### 네이버 뉴스 크롤링 최종
- 섹션 추출 ~ sub 페이지 추출

#### (1) + (2) + (3)
(1) 섹션 메뉴와 섹션별 url 추출      
    - section, link  
    - section_menu_df  
(2) 섹션별 헤드라인(topic) 제목과 url 추출  
    - topic, url, section  
    - all_topic_df  
(3) 섹션 헤드라인별 sub 페이지 기사 내용 추출  
    - 언론사, 제목, 날짜/시간, 기사내용, 작성자  
    - paper, title, datetime, writing, writer  
    - all_sub_news_info_df   

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from urllib.request import urlopen
import pandas as pd
import bs4
import requests

In [3]:
url = 'https://news.naver.com/'
html = urlopen(url)
# 파서 객체 생성
bs_obj = bs4.BeautifulSoup(html, 'html.parser')

#### (1) 네이버 뉴스 섹션 메뉴와 url 추출

In [4]:
ul = bs_obj.find('ul', {'class':'Nlnb_menu_list'})
# ul

In [5]:
# 수집된 데이터로 df 구성

lis = ul.select('li')

section = []
link = []

# 리스트에 추가 
for li in lis:
    a_tag = li.find('a')
    section.append(a_tag.text)
    link.append(a_tag['href'])

section_menu_df = pd.DataFrame({'section':section, 'link':link})
section_menu_df

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...
8,신문보기,https://news.naver.com/newspaper/home?viewType=pc
9,오피니언,https://news.naver.com/opinion/home


#### (2) 섹션별 헤드라인(topic) 제목과 url 추출

In [6]:
# 해당 섹션에 대한 모든 topic 크롤링하는 함수 생성
def get_topic(url, section):
    # 접속 및 결과 파싱
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')

    # topic_list 추출
    topic_list = bs_obj.findAll('div', {'class':'sh_text'})

    # 제목과 링크 추출해서 리스트에 추가
    topic_title = []
    topic_link = []

    for topic in topic_list:    
        try :
            a = topic.find('a')
            topic_title.append(a.text)
            topic_link.append(a['href'])
        except:
            print('오류 발생')
            # pass

    # 딕셔너리 만들어서 반환
    return ({'topic':topic_title, 'url':topic_link, 'section':section})

In [7]:
# 모든 섹션에 대한 모든 topic 데이터프레임 생성
all_topic_df = pd.DataFrame({
    'topic':[],
    'url': [],
    'section': []
})

for i in range(1, 7) :
    df = pd.DataFrame(get_topic(section_menu_df['link'][i],  section_menu_df['section'][i]))
    all_topic_df = pd.concat([all_topic_df, df], axis=0, ignore_index=True)

# all_topic_df

#### (3) 섹션 헤드라인별 sub 페이지 기사 내용 추출
- 언론사, 제목, 날짜/시간, 기사내용, 작성자

In [8]:
# sub 페이지 기사 내용 추출하는 함수
def get_sub_news_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    # 빈 리스트 생성
    paper_list, title_list, datetime_list, writing_list, writer_list = [ [] for _ in range(5) ]
    # for _ in range(5) : 언더바(_) (변수 없이 반복만 수행)
    
    try :
        
        res = requests.get(url, headers=headers)
        html = res.text
        bs_obj = bs4.BeautifulSoup(html, 'html.parser')
        # 접속    
    
        # 언론사, 기사제목, 작성 날짜 시간, 기사 내용, 작성자 리스트 생성
        paper_list.append(bs_obj.find('a', {'class':'media_end_head_top_logo'}).select_one('img')['title'])
        title_list.append(bs_obj.find('div', {'class' : 'media_end_head_title'}).select_one('span').text)
        datetime_list.append(bs_obj.find('span', {'class' : 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}).text)
        writing_list.append(bs_obj.select_one('#dic_area').text.replace('\n', ''))
        writer_list.append(bs_obj.find('span',{'class':'byline_s'}).text)

    except:
        print("오류 발생")
        # pass


    # 딕셔너리로 만들어서 반환
    sub_news_dict = {
        'paper':paper_list,
        'title':title_list,
        'datetime':datetime_list,
        'writing':writing_list,
        'writer':writer_list
    }

    return sub_news_dict

In [9]:
# 모든 sub 페이지 내용 저장하는 데이터프레임 생성해서 최종 데이터프레임 생성
all_sub_news_info_df = pd.DataFrame({
    'paper': [],
    'title': [],
    'datetime': [],
    'writing': [],
    'writer': []
})

for url in all_topic_df['url']:
    df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df = pd.concat([all_sub_news_info_df, df], axis=0, ignore_index=True)

all_sub_news_info_df

,paper,title,datetime,writing,writer
0,동아일보,“이재명 체포안 논의땐 당 분열”… 민주당의 고민,2023.09.12. 오후 7:13,이재명 더불어민주당 대표가 쌍방울그룹 대북송금 의혹 관련 조사를 받기 위해 12일 ...,강성휘 기자 yolo@donga.com
1,SBS,"민주당, 태영호 징계 추진…""이재명 단식 중 행패, 묵과 못해""",2023.09.12. 오후 2:00,▲ 지난 7일 이재명 대표에게 항의하는 태영호 의원민주당은 단식 중인 이재명 대표를...,장민성 기자 ms@sbs.co.kr
2,뉴시스,"국힘, '대장동 허위 인터뷰 방송' 김어준·주진우·최경영 명예훼손 고발",2023.09.13. 오전 10:47,"국힘, 추가 고발 예고…""자극적인 표현으로 강하게 오인하도록 한 경우""[서울=뉴시스...",이재우 기자(ironn108@newsis.com)
3,연합뉴스,"한총리, 프라하서 체코 총리와 회담…""에너지·인프라 협력강화""",2023.09.13. 오전 8:44,"체코 상원 의장에게 ""한국 기업 어려움 해결에 관심 가져달라""피알라 체코 총리와 악...",한혜원(hye1@yna.co.kr)
4,문화일보,"尹, 정당한 교권 보장 ‘가이드라인’ 지시",2023.09.12. 오전 11:49,‘교권 회복’ 방안은… 박대출(오른쪽 두 번째) 국민의힘 정책위의장이 12일 오전 ...,최지영 기자(goodyoung17@munhwa.com)
5,한겨레,"윤 대통령, ‘교권보호 4법’ 처리 촉구…“교육 정상화 신속하게”",2023.09.12. 오전 11:29,초중등교육법·유아교육법·교원지위법·교육기본법 꼽아윤석열 대통령이 12일 용산 대통령...,배지현 기자 beep@hani.co.kr
6,한국경제,"與 ""'김건희법' 추진 변함없다…개 식용 종식 실천할 때""",2023.09.13. 오전 10:06,"'김건희법 제정 불투명' 복수 언론 보도박대출 ""그런 방침 정한 적 없다"" 일축김건...",홍민성 한경닷컴 기자 mshong@hankyung.com
7,머니투데이,"'단식 14일차' 이재명, 당 대표실로 단식장 이동…""결연한 의지""",2023.09.13. 오전 11:30,[the300](서울=뉴스1) 송원영 기자 = 이재명 더불어민주당 대표가 12일 저...,차현아 기자 (chacha@mt.co.kr)
8,중앙일보,"""개념 없는 연예인""…'자우림 김윤아' 저격한 김기현 여당 대표",2023.09.12. 오후 4:44,김기현 국민의힘 대표가 12일 오후 서울 중구 한국관광공사 서울센터에서 열린 문...,현예슬 기자 hyeon.yeseul@joongang.co.kr
9,문화일보,최대규모 국군의 날 시가행진… 美특전사 참가 한미동맹 과시,2023.09.13. 오후 12:01,9월 26일 병력 6700명 장비 340대 투입오는 26일 열리는 건군 75주년 국...,정충신 선임기자(csjung@munhwa.com)


In [10]:
# 최종 결과 파일로 저장
all_sub_news_info_df.to_csv('./crawl_data/naver_sub_news_section.csv', index=0)

In [11]:
all_sub_news_info_df = pd.read_csv('./crawl_data/naver_sub_news_section.csv')
all_sub_news_info_df

,paper,title,datetime,writing,writer
0,동아일보,“이재명 체포안 논의땐 당 분열”… 민주당의 고민,2023.09.12. 오후 7:13,이재명 더불어민주당 대표가 쌍방울그룹 대북송금 의혹 관련 조사를 받기 위해 12일 ...,강성휘 기자 yolo@donga.com
1,SBS,"민주당, 태영호 징계 추진…""이재명 단식 중 행패, 묵과 못해""",2023.09.12. 오후 2:00,▲ 지난 7일 이재명 대표에게 항의하는 태영호 의원민주당은 단식 중인 이재명 대표를...,장민성 기자 ms@sbs.co.kr
2,뉴시스,"국힘, '대장동 허위 인터뷰 방송' 김어준·주진우·최경영 명예훼손 고발",2023.09.13. 오전 10:47,"국힘, 추가 고발 예고…""자극적인 표현으로 강하게 오인하도록 한 경우""[서울=뉴시스...",이재우 기자(ironn108@newsis.com)
3,연합뉴스,"한총리, 프라하서 체코 총리와 회담…""에너지·인프라 협력강화""",2023.09.13. 오전 8:44,"체코 상원 의장에게 ""한국 기업 어려움 해결에 관심 가져달라""피알라 체코 총리와 악...",한혜원(hye1@yna.co.kr)
4,문화일보,"尹, 정당한 교권 보장 ‘가이드라인’ 지시",2023.09.12. 오전 11:49,‘교권 회복’ 방안은… 박대출(오른쪽 두 번째) 국민의힘 정책위의장이 12일 오전 ...,최지영 기자(goodyoung17@munhwa.com)
5,한겨레,"윤 대통령, ‘교권보호 4법’ 처리 촉구…“교육 정상화 신속하게”",2023.09.12. 오전 11:29,초중등교육법·유아교육법·교원지위법·교육기본법 꼽아윤석열 대통령이 12일 용산 대통령...,배지현 기자 beep@hani.co.kr
6,한국경제,"與 ""'김건희법' 추진 변함없다…개 식용 종식 실천할 때""",2023.09.13. 오전 10:06,"'김건희법 제정 불투명' 복수 언론 보도박대출 ""그런 방침 정한 적 없다"" 일축김건...",홍민성 한경닷컴 기자 mshong@hankyung.com
7,머니투데이,"'단식 14일차' 이재명, 당 대표실로 단식장 이동…""결연한 의지""",2023.09.13. 오전 11:30,[the300](서울=뉴스1) 송원영 기자 = 이재명 더불어민주당 대표가 12일 저...,차현아 기자 (chacha@mt.co.kr)
8,중앙일보,"""개념 없는 연예인""…'자우림 김윤아' 저격한 김기현 여당 대표",2023.09.12. 오후 4:44,김기현 국민의힘 대표가 12일 오후 서울 중구 한국관광공사 서울센터에서 열린 문...,현예슬 기자 hyeon.yeseul@joongang.co.kr
9,문화일보,최대규모 국군의 날 시가행진… 美특전사 참가 한미동맹 과시,2023.09.13. 오후 12:01,9월 26일 병력 6700명 장비 340대 투입오는 26일 열리는 건군 75주년 국...,정충신 선임기자(csjung@munhwa.com)
